# 基于CSV文件的RAG（检索增强生成）系统

这段代码实现了一个基本的Retrieval-Augmented Generation（RAG）系统，用于处理和查询CSV文档。系统将文档内容编码到向量数据库中，然后可以查询该向量数据库以检索相关信息。

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["MINIMAX_GROUP_ID"] = os.getenv("MINIMAX_GROUP_ID")
os.environ["MINIMAX_API_KEY"] = os.getenv("MINIMAX_API_KEY")

import pandas as pd

from langchain_qdrant import QdrantVectorStore
from langchain_community.chat_models import MiniMaxChat
from langchain_community.embeddings import MiniMaxEmbeddings
from langchain_community.document_loaders import CSVLoader

In [18]:
file_path = ('../data/review.csv') # insert the path of the csv file
data = pd.read_csv(file_path, encoding='gbk')

data.head()

,二级标签,健康度分类,网络来源,发布者,标题,内容,ocr内容,车型频次,链接,调性
0,产品报道,综合类,今日头条,优雅风声sNCb6L,优雅风声sNCb6L发布的小视频,优雅风声sNCb6L发布的小视频,丰田日产 No.2 迈巴赫奔驰凯迪拉克 No.3 No.4 LAND> -ROVER 路虎林...,NaN,http://www.toutiao.com/item/7410124569756945727,中性
1,售后服务,视频类,抖音,陕西汽车24小时救援,#道路救援拖车 #应急救援 #一拖二清障车 #工程机械设备 #随时待命,#道路救援拖车 #应急救援 #一拖二清障车 #工程机械设备 #随时待命,综合力车有业 X70Plus送往修理厂帮忙给伙计救援一台捷途,NaN,https://www.douyin.com/video/7410077677136203062,中性
2,品牌形象,视频类,抖音,潘大哥,NaN,NaN,人王球热飞技术奇寺瑞奇瑞汽车众鑫瑞店交车区,奇瑞汽车(1),https://www.douyin.com/note/7410077707960061193,正面
3,产品报道,综合类,百家号,极速车天下,9月8款新能源SUV即将上市，哪款最合你胃口？,进入“金九银十”这一传统的购车旺季后，各大车企纷纷推出重磅新车。据统计，今年9月将有8款新能...,NaN,NaN,http://baijiahao.baidu.com/s?id=18090824920482...,正面
4,销量业绩,综合类,微头条,创业阿甘,科创早讯 2024-09-03 星期二 1、鸿蒙智行8月全系交付新车33699辆（e公司） ...,科创早讯 2024-09-03 星期二 1、鸿蒙智行8月全系交付新车33699辆 2、泸州老...,NaN,NaN,http://www.toutiao.com/item/1809100378882112,正面


In [19]:
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()
docs

[Document(metadata={'source': '../data/review.csv', 'row': 0}, page_content='二级标签: 产品报道\n健康度分类: 综合类\n网络来源: 今日头条\n发布者: 优雅风声sNCb6L\n标题: 优雅风声sNCb6L发布的小视频\n内容: 优雅风声sNCb6L发布的小视频\nocr内容: 丰田日产 No.2 迈巴赫奔驰凯迪拉克 No.3 No.4 LAND> -ROVER 路虎林肯捷豹 No.5 STUTTGART 兰博基尼保时捷 No.6 雷克萨斯讴歌长安 No.7 英菲尼迪沃尔沃奇瑞 INFINITI No.8 三菱科尔维特五菱宏光 No.9 柯尼塞格法拉利 apollo 布加迪测测你认识多少车标阿波罗世爵无敌了! No.10 居然全认识\n车型频次: \n链接: http://www.toutiao.com/item/7410124569756945727\n调性: 中性'),
 Document(metadata={'source': '../data/review.csv', 'row': 1}, page_content='二级标签: 售后服务\n健康度分类: 视频类\n网络来源: 抖音\n发布者: 陕西汽车24小时救援\n标题: #道路救援拖车 #应急救援 #一拖二清障车 #工程机械设备 #随时待命\n内容: #道路救援拖车 #应急救援 #一拖二清障车 #工程机械设备 #随时待命\nocr内容: 综合力车有业 X70Plus送往修理厂帮忙给伙计救援一台捷途\n车型频次: \n链接: https://www.douyin.com/video/7410077677136203062\n调性: 中性'),
 Document(metadata={'source': '../data/review.csv', 'row': 2}, page_content='二级标签: 品牌形象\n健康度分类: 视频类\n网络来源: 抖音\n发布者: 潘大哥\n标题: \n内容: \nocr内容: 人王球热飞技术奇寺瑞奇瑞汽车众鑫瑞店交车区\n车型频次: 奇瑞汽车(1)\n链接: https://www.douyin.com/note/7410077707960061193\n调性: 正面'),
 Doc

In [20]:
embeddings = MiniMaxEmbeddings()
vectorstore = QdrantVectorStore.from_documents(
    docs,
    embeddings,
    url="http://localhost:6333/",
    prefer_grpc=True,
    collection_name="006"
)

In [21]:
retriever = vectorstore.as_retriever()

In [25]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

llm = MiniMaxChat()

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
answer= rag_chain.invoke({"input": "发布者: 落寞爱剪辑，写的标题是什么内容？"})
answer['answer']

'发布者落寞爱剪辑的标题是："落寞爱剪辑发布的小视频"，内容涉及奇瑞的置换政策，提到有旧车的车主可以参与置换，享受抵扣优惠，并鼓励点击视频链接联系了解详情。'